In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import pandas as pd
from langchain_community.embeddings.voyageai import VoyageEmbeddings
from langchain_voyageai import VoyageAIEmbeddings
from langchain_text_splitters import RecursiveJsonSplitter
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain_pinecone import PineconeVectorStore

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
os.environ["VOYAGE_API_KEY"] = " "
os.environ["PINECONE_API_KEY"] = "pcsk_3TzW3X_SAqi8rhAzX9EWyCgDAgWAmhxkeJiDK7J7W2ahSjmUvd3Gm66YgLvsFwJxW2cc3c"

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LoadParquet").getOrCreate()

parquet_path = "/home/misha/Desktop/BIG DATA PROJECT/pubmed_data-20250330T183136Z-001./pubmed_data/df_pubmed.parquet"

df_pubmed = spark.read.parquet(parquet_path)

25/04/13 22:59:09 WARN Utils: Your hostname, misha resolves to a loopback address: 127.0.1.1; using 192.168.1.101 instead (on interface wlp44s0)
25/04/13 22:59:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 22:59:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from langchain.schema import Document

sampled_rows = df_pubmed.limit(700).collect()

documents = [
    Document(
        page_content=row['Abstract'] if row['Abstract'] else "",
        metadata={"title": row['Title'] if row['Title'] else ""}
    )
    for row in sampled_rows
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200)
docs_split = text_splitter.split_documents(documents)

In [ ]:
texts = [doc.page_content for doc in docs_split]
metadatas = [doc.metadata for doc in docs_split]

25/04/13 22:59:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
### LOAD THE FILE CONTENTS INTO PINECONE - RUN THIS ONLY ONCE
embeddings = VoyageAIEmbeddings(
    model="voyage-3-large", 
    output_dimension=1024
)

index_name = ""

vectorstore = PineconeVectorStore.from_texts(
    texts=texts,  
    embedding=embeddings,
    metadatas=metadatas,
    index_name=index_name)

print("Done")

Successfully uploaded documents to Pinecone


In [ ]:
vectorstore_abstracts = PineconeVectorStore(
    index_name="1024", 
    embedding=embeddings
)
retriever = vectorstore_abstracts.as_retriever(search_kwargs={"k": 3})

In [ ]:
### Your question here
retriever.invoke(" ")